In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.
%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)

In [ ]:
import tsdm

In [ ]:
ds = tsdm.datasets.KIWI_RUNS()

In [ ]:
ds.timeseries

In [ ]:
from tsdm.datasets import TimeSeriesCollection

In [ ]:
str(tuple(ds.timeseries.shape))

In [ ]:
TSC = TimeSeriesCollection(
    ds.index,
    ds.timeseries,
    ds.metadata,
    time_features=ds.time_features,
    value_features=ds.value_features,
    metadata_features=ds.metadata_features,
)

In [ ]:
TSC.__dataclass_fields__

In [ ]:
import dataclasses

In [ ]:
dataclasses.is_dataclass(type(TSC))

In [ ]:
dir(TSC)